<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkST0151ENSkillsNetwork20531532-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>



#### Import the required libraries we need for the lab.


In [ ]:
import piplite
await piplite.install(['numpy'],['pandas'])
await piplite.install(['seaborn'])

In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Read the dataset in the csv file from the URL


In [ ]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv'
resp = await fetch(URL)
boston_url = io.BytesIO((await resp.arrayBuffer()).to_py())

In [ ]:
boston_df=pd.read_csv(boston_url)

# Task 1: Become familiar with the dataset

In [ ]:
boston_df.head()

#### Add your code below following the instructions given in the course to complete the peer graded assignment


# Task 2: Generate Descriptive Statistics and Visualizations

## Q1 : For the "Median value of owner-occupied homes" provide a boxplot

In [ ]:
sns.boxplot(y='MEDV', data = boston_df)
plt.title("Boxplot of MEDV")
plt.ylabel("MEDV")
plt.show()

## Q2 : Provide a  bar plot for the Charles river variable

In [ ]:
sns.catplot(x='CHAS',kind='count', data=boston_df)
plt.show()

## Q3 : Provide a boxplot for the MEDV variable vs the AGE variable.

#### *(Discretize the age variable into three groups of 35 years and younger, between 35 and 70 years and 70 years and older)*

In [ ]:
# Discretizing AGE variable
boston_df.loc[(boston_df['AGE'] <= 35), 'AGE_GRP'] = '35 years and younger'
boston_df.loc[(boston_df['AGE'] > 35)&(boston_df['AGE'] < 70), 'AGE_GRP'] = 'between 35 and 70 years'
boston_df.loc[(boston_df['AGE'] >= 70), 'AGE_GRP'] = '70 years and older'
# Plotting Boxplot
sns.boxplot(x='AGE_GRP', y='MEDV', order = ['35 years and younger', 'between 35 and 70 years', '70 years and older'], data = boston_df)
plt.show()

## Q4 : Provide a scatter plot to show the relationship between Nitric oxide concentrations and the proportion of non-retail business acres per town. What can you say about the relationship?

In [ ]:
sns.scatterplot(x='INDUS', y='NOX', data=boston_df)
plt.show()

## Q5 : Create a histogram for the pupil to teacher ratio variable

In [ ]:
sns.displot(boston_df['PTRATIO'], color='green', kde=True)
plt.show()

# Task 3: Use the appropriate tests to answer the questions provided.

## Q1 : Is there a significant difference in median value of houses bounded by the Charles river or not? (T-test for independent samples)

$H₀$ : No difference in MEDV between CHAS groups

$α$ = 0.05

In [ ]:
t_stat, p_value_tt = scipy.stats.ttest_ind(boston_df[boston_df['CHAS'] == 1]['MEDV'],
                                        boston_df[boston_df['CHAS'] == 0]['MEDV'], 
                                        equal_var = False)
print("P_value : ", p_value_tt)

$p = 0.00356 < 0.05$ → We reject the null hypothesis.

There is a statistically significant difference in median house values between houses near the Charles River and those not near it. Houses located near the river tend to have higher median values.

## Q2 : Is there a difference in Median values of houses (MEDV) for each proportion of owner occupied units built prior to 1940 (AGE)? (ANOVA)

$H₀$ : The average MEDV is the same across all AGE groups

$α$ = 0.05

In [ ]:
f_statistic_anova, p_value_anova = scipy.stats.f_oneway(boston_df[boston_df['AGE_GRP'] == '35 years and younger']['MEDV'],
                                            boston_df[boston_df['AGE_GRP'] == 'between 35 and 70 years']['MEDV'],
                                            boston_df[boston_df['AGE_GRP'] == '70 years and older']['MEDV'])
print("P-Value : ",p_value_anova)

$p < 0.05$ → We reject the null hypothesis.

There is a statistically significant difference in median house values between at least two age groups.

## Q3 : Can we conclude that there is no relationship between Nitric oxide concentrations and proportion of non-retail business acres per town? (Pearson Correlation)

$H₀$ : There is no linear correlation between NOX (nitric oxide concentration) and INDUS (percentage of industrial land).

$α$ = 0.05

In [ ]:
f_statistic_pearson, p_value_pearson = scipy.stats.pearsonr(boston_df['NOX'], boston_df['INDUS']) 
print("P-Value : ",p_value_pearson)

$p < 0.05$ → We reject the null hypothesis.

There is a strong and statistically significant positive correlation between NOX and INDUS. This suggests that more industrial areas tend to have higher air pollution levels.

## Q4 : What is the impact of an additional weighted distance  to the five Boston employment centres on the median value of owner occupied homes? (Regression analysis)

$H₀$ : The distance to employment centers (DIS) has no effect on median house value (MEDV).

$α$ = 0.05

In [ ]:
X = boston_df['DIS']
y = boston_df['MEDV']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Print out the statistics
model.summary()

$p < 0.05$ → We reject the null hypothesis.

DIS has a statistically significant and positive effect on MEDV.
However, the R-squared value of 0.0625 indicates that DIS alone is statistically significant, but not a strong predictor on its own.

**Answer to the question :**

For each additional unit of distance, the median value of houses increases by about *$1091.6*

# Author
## Aymane BENCHAD